In [1]:
import torch
import torch.nn as nn
from imports import *

In [2]:
input_text = []
input_text1 = 'This is a keyboard'
input_text2 = 'A nice coffee cup'
input_text.append(input_text1)
input_text.append(input_text2)

# for a single text
token_ids_single, input_embeddings_single = tokenize_text(input_text[0])
print("\nFor a single text:")
print(input_embeddings_single.shape)
print(input_embeddings_single)
print(token_ids_single)

# for batched processing
token_ids, input_embeddings = tokenize_batch(input_text)
print(input_embeddings.shape)
print(input_embeddings)
print(token_ids)


For a single text:
torch.Size([1, 4, 512])
tensor([[[-2.4624, -0.6219,  0.3427,  ...,  0.4021, -0.4395,  0.0513],
         [-2.2234, -0.0615, -0.3485,  ...,  2.0056,  0.6611,  0.1068],
         [ 0.4423,  0.7279,  1.3436,  ...,  1.3718,  0.1375,  1.7969],
         [ 1.8609,  0.4966, -0.2507,  ...,  1.1169,  0.7286, -2.2299]]],
       grad_fn=<EmbeddingBackward0>)
[1212, 318, 257, 10586]
torch.Size([2, 4, 512])
tensor([[[-2.4624, -0.6219,  0.3427,  ...,  0.4021, -0.4395,  0.0513],
         [-2.2234, -0.0615, -0.3485,  ...,  2.0056,  0.6611,  0.1068],
         [ 0.4423,  0.7279,  1.3436,  ...,  1.3718,  0.1375,  1.7969],
         [ 1.8609,  0.4966, -0.2507,  ...,  1.1169,  0.7286, -2.2299]],

        [[ 0.7492,  0.7029,  1.8907,  ...,  1.1608,  0.6677,  0.9957],
         [ 0.6290, -1.2532,  0.7946,  ...,  1.1605,  0.3518,  0.7011],
         [-0.4044, -0.7931,  1.1574,  ..., -0.4258,  0.3240,  1.1428],
         [ 0.7365,  0.9845,  0.7070,  ...,  0.2339,  0.2194, -0.1703]]],
       grad_f

In [3]:
encoded_embedding = positional_encoding(input_embeddings=input_embeddings)
encoded_embedding

tensor([[[-2.4624,  0.3781,  0.3427,  ...,  1.4021, -0.4395,  1.0513],
         [-1.3819,  0.5081,  0.4535,  ...,  3.0056,  0.6611,  1.1068],
         [ 1.3516,  0.3770,  2.3018,  ...,  2.3718,  0.1375,  2.7969],
         [ 2.0021, -0.4729,  0.0920,  ...,  2.1169,  0.7286, -1.2299]],

        [[ 0.7492,  1.7029,  1.8907,  ...,  2.1608,  0.6677,  1.9957],
         [ 1.4705, -0.6835,  1.5966,  ...,  2.1605,  0.3518,  1.7011],
         [ 0.5049, -1.1440,  2.1155,  ...,  0.5742,  0.3240,  2.1428],
         [ 0.8776,  0.0150,  1.0497,  ...,  1.2339,  0.2194,  0.8297]]],
       grad_fn=<AddBackward0>)

## Moving on to the encoder block